## Multiple object tracking and re-identification with DeepSORT · Object database

In [11]:
# !pip install ultralytics
# !pip install deep-sort-realtime

#### Class to write videos

In [193]:
import os, cv2

class Video:

    def __init__(self, codec:str='MP4V', fps:int=3, shape:tuple=(854, 480), overwrite=False):
        self.codec = codec; self.fps = fps; self.shape = shape
        self.overwrite = overwrite

    def writer(self, path):
        if not self.overwrite and os.path.exists(path):
            print(f'ANNOTATE VIDEO TIMESTAMP FAILED. FILE ALREADY EXISTS · FILE-PATH: {path}')
            return False
        fourcc = cv2.VideoWriter_fourcc(*self.codec)
        return cv2.VideoWriter(path, fourcc, self.fps, self.shape)

    
#### OPEN VIDEO FILE WRITER · Method #1
# video_path = "output.mp4"
# fps, shape = get_video_metadata(video_path, transform=None)
# shape = (shape[1], shape[0]) # witdth, height
# overwrite = True
# video = Video(fps=fps, shape=shape, overwrite=overwrite)
# writer = video.writer(path=video_path)

# writer.release(); cv2.destroyAllWindows()

#### Function to get basic metadata from video file:
    fps: frames per second (FPS) of video file
    shape: shape of first frame

In [194]:
def get_video_metadata(video_path, transform=None):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS) # get the fps
    _, frame = cap.read() # read the first frame
    if transform is not None: # custom transformation
        frame = transform(frame)
    shape = frame.shape; # get the shape
    cap.release(); cv2.destroyAllWindows()
    return fps, shape

#### Function to open video file writer

In [195]:
import cv2

def create_video_writer(video_cap, output_filename):

    # grab the width, height, and fps of the frames in the video stream.
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    # initialize the FourCC and a video writer object
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    writer = cv2.VideoWriter(output_filename, fourcc, fps,
                             (frame_width, frame_height))

    return writer

#### Format output of YOLO object detection model from python's `ultralitycs` module

In [196]:
def formatted_yolo_detection(detection, class_names=None):
    """
    Formats the YOLO detection results.

    Args:
        detection (object): The detection object.
        class_names (dict): Dict of class names by class id.

    Returns:
        list: Formatted detection results.
    """
    formatted_detection = []

    for data in detection.boxes.data.tolist():
        # Get the bounding box and the class id
        xmin, ymin, xmax, ymax = int(data[0]), int(data[1]), int(data[2]), int(data[3])

        # Extract the confidence (i.e., probability) associated with the prediction
        confidence = data[4]

        # Get class id and name
        class_id = int(data[5])
        class_name = class_names[class_id] if class_names is not None else None

        # Add standard format detections
        formatted_detection.append([class_id, class_name, confidence, [xmin, ymin, xmax, ymax]])

    return formatted_detection


### Function to process object identification output (post processing)

In [197]:
# code for tracking and detection from: https://www.thepythoncode.com/article/real-time-object-tracking-with-yolov8-opencv

import datetime, pandas as pd
from IPython.display import clear_output as co
from ultralytics import YOLO
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort
import asyncio

def tracking_reid(
    video_path,
    confidence_threshold=0.3,
    objects_allowed=None,
    max_frames=10,
    post_processing_function=None,
    post_processing_args={},
    proccess_each=None,
    frame_annotator=None,
    to_video_path=None,
    generator=False
):
    
    # initialize YOLO object detection model
    model = YOLO("yolov8n.pt")

    # Get class names from model
    class_names = model.names

    # initialize DeepSORT real-time tracker
    deepsort = DeepSort(max_age=50)
        
    # initialize the video capture object
    video_cap = cv2.VideoCapture(video_path)
    
    # check if video capture is a live http image stream
    is_video_stream = video_path.startswith('http')

    # total frames of video file
    total_frames = None if is_video_stream else int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT)) # if capture is from video file

    if to_video_path is not None:
        # Get the frames per second (fps)
        fps = video_cap.get(cv2.CAP_PROP_FPS)

        # Get the frame dimensions (shape)
        w = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Video writer instance
        video = Video(codec='MP4V', fps=fps, shape=(w, h), overwrite=True)
        WRITER = video.writer(to_video_path)

    # initialize set for track ids 
    unique_track_ids = set()

    # initialize post processing output list
    post_processing_output = []
    
    i = -1
    while True:

        # update number of frames processed or skipped
        i += 1
        
        # break loop if `max_frames` are processed
        if max_frames is not None and max_frames == i:
            break

        # current date and time
        start = datetime.datetime.now()
        
        # datetime as string rounded to seconds
        timestamp = str(start)[:19]
        
        # read video frame
        ret, frame = video_cap.read()
        if not ret:
            break
        
        # continue if frame index `i` is not a multiple of `process_each`. never continues for first frame
        if proccess_each is not None and i % proccess_each != 0:
            continue

        ######################################
        # RUN DETECTION · Obs. Choose standard model method for prediction and wrap models that use other methods before passing then to the function.

        # run the YOLO model on the frame
        yolo_detection = model(frame)[0]
        
        # formatted yolo detections
        detections = formatted_yolo_detection(yolo_detection, class_names=class_names)

        # initialize list for tracker input
        tracker_input = []
        
        # set up input for tracker from detections
        for det in detections:
            
            # get detected object attributes
            class_id, class_name, confidence, bbox = det
            
            # filter out weak detections by ensuring the 
            # confidence is greater than the minimum confidence
            if float(confidence) < confidence_threshold:
                continue
                
            # filter out unwanted objects  
            if objects_allowed is not None and class_name not in objects_allowed:
                continue

            # if the confidence is greater than the minimum confidence,
            # get the bounding box and the class id
            xmin, ymin, xmax, ymax = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
            
            # add the bounding box (x, y, w, h), confidence and class id to the results list
            tracker_input.append([[xmin, ymin, xmax - xmin, ymax - ymin], confidence, class_id])

        ######################################
        # RUN TRACKING

        # update the tracker with the new detections
        tracks = deepsort.update_tracks(tracker_input, frame=frame)
        
        # initialize list for formatted tracker output
        tracking = []

        # list tracking result
        for track in tracks:
        
            # if the track is not confirmed, ignore it
            if not track.is_confirmed():
                continue
            
            # get attributes of tracked object
            track_id = track.track_id
            class_label = track.det_class
            class_name = class_names[class_label]
            confidence = track.det_conf
            bbox = track.to_ltrb()

            # append attributes of tracked objects
            tracking.append([track_id, class_label, class_name, confidence, bbox, start])
        
        ######################################
        # GET NEWLY IDENTIFIED OBJECTS
        
        # initialize list for newly detected objects
        new_objects = []

        # loop over the formatted tracks and get newly identified objects
        for track in tracking:

            # get track attributes
            track_id, class_label, class_name, confidence, bbox, timestamp = track

            # check if track ID is unique
            if track_id not in unique_track_ids:

                # prepare record of newly identified object
                record = {
                    'class_label': class_label,
                    'class_name': class_name,
                    'confidence': confidence,
                    'timestamp': timestamp,
                    'track_id': track_id,
                    'bbox': list(bbox),
                }

                # append record to list of new objects
                new_objects.append(record)

                # add the tracked object ID to the set of unique track IDs
                unique_track_ids.add(track_id)

        ######################################
        # PROCESS RESULT
        
        # end time to compute the fps
        end = datetime.datetime.now()
        
        if frame_annotator is not None:
            annotated_frame = frame_annotator(frame, detections, tracking, new_objects, start, end)
            
        if to_video_path is not None:
            selected_frame = frame if frame_annotator is None else annotated_frame
            WRITER.write(selected_frame)
                
        # call arbitrary post processing function on frame and detection & tracking outputs
        if post_processing_function is not None:
            post_processing_output.append(post_processing_function(frame, detections, tracking, new_objects, start, end, **post_processing_args))

        # report progress and time to process the current frame
        # if total_frames is not None:
            # co(True); print(f"Time to process frame {i}/{total_frames}: {(end - start).total_seconds() * 1000:.0f} milliseconds")

        if generator:
            selected_frame = frame if frame_annotator is None else annotated_frame
            ret, buffer = cv2.imencode('.jpg', selected_frame)
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + buffer.tobytes() + b'\r\n')
        
    if to_video_path is not None:
        # Optional · release video writer
        WRITER.release() # run after writing is finished

    # release video capture
    video_cap.release()
    cv2.destroyAllWindows()
    
    # return post processing results
    return post_processing_output


## Multiple object tracking and re-identification with DeepSORT

### Process detection and re-identification results · Example usage

Define system paths

In [198]:
# system paths
folder = '../Dados/Demos/smartphone-video-samples/'
to_folder = '../Dados/Demos/tracking-id-db/'
file_name = 'VID_20230515_125317.mp4'

video_path = folder + file_name
to_video_path = to_folder + file_name

#### Function to write frame and results to video file · Example usage

In [199]:
# set up color scheme
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
WHITE = (255, 255, 255)

def write_demo(frame, detections, tracking, new_objects, process_start, process_end):

    # loop over the formatted tracks and get newly identified objects
    for track in tracking:

        # get track attributes
        track_id, class_label, class_name, confidence, bbox, timestamp = track

        # get pixel values from track bounding box
        xmin, ymin, xmax, ymax = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
        
        # draw the bounding box and the track id
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), GREEN, 2)
        cv2.rectangle(frame, (xmin, ymin - 40), (xmin + 40, ymin), GREEN, -1)
        cv2.putText(frame, str(track_id), (xmin + 5, ymin - 8), cv2.FONT_HERSHEY_SIMPLEX, 1, WHITE, 2)
        
    # calculate the frame per second and draw it on the frame
    fps = f"FPS: {1 / (process_end - process_start).total_seconds():.2f}"
    cv2.putText(frame, fps, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, BLUE, 8)
    
    return frame
    # write annotated frame to video file
    # WRITER.write(frame)

    
#### Run pipeline · Example usage

post_processing_output = tracking_reid(
    video_path,
    confidence_threshold=0.5,
    max_frames=20,
    post_processing_function=None,
    proccess_each=None,
    frame_annotator=write_demo,
    to_video_path=to_video_path,
)


#### Show post processing outputs

In [44]:
post_processing_output

[None, None, None, None, None, None, None, None]

---
## Example usage with other sample functions

#### Function to display frame of video demonstration of detection and tracking with re-identification

In [200]:
import matplotlib.pyplot as plt
from IPython.display import clear_output as co

# set up color scheme
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
WHITE = (255, 255, 255)

def show_demo(frame, detections, tracking, new_objects, process_start, process_end):

    # Convert BGR image to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # loop over the formatted tracks and get newly identified objects
    for track in tracking:

        # get track attributes
        track_id, class_label, class_name, confidence, bbox, timestamp = track

        # get pixel values from track bounding box
        xmin, ymin, xmax, ymax = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])

        # draw the bounding box and the track id
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), GREEN, 2)
        cv2.rectangle(frame, (xmin, ymin - 40), (xmin + 40, ymin), GREEN, -1)
        cv2.putText(frame, str(track_id), (xmin + 5, ymin - 8), cv2.FONT_HERSHEY_SIMPLEX, 1, WHITE, 2)

    # calculate the frame per second and draw it on the frame
    fps = f"Processing FPS: {1 / (process_end - process_start).total_seconds():.2f}"
    cv2.putText(frame, fps, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, BLUE, 8)
    
    # clear last frame screen output
    co(True)
    
    # show the frame to our screen
    plt.imshow(frame);
    plt.xticks([]); plt.yticks([])
    plt.show()
    
    # cv2.imshow("Frame", frame)
    # cv2.waitKey(1) == ord("q")
    # plt.show()

    
#### Run pipeline · Example usage

post_processing_output = tracking_reid(
    video_path,
    confidence_threshold=0.5,
    max_frames=200,
    post_processing_function=show_demo,
    proccess_each=10,
)

#### Function to gather id and info of newly identified objects from frame

In [ ]:
import numpy as np, pandas as pd
from IPython.display import clear_output as co

def gather_unique_objects(frame, detections, tracking, new_objects, process_start, process_end):

    # extend list of unique objects with new objects
    return new_objects
    
    
#### Run pipeline · Example usage

# initialize set for track ids 
unique_track_ids = set()

post_processing_output = tracking_reid(
    video_path,
    confidence_threshold=0.5,
    max_frames=20,
    post_processing_function=gather_unique_objects,
    proccess_each=None,
)

# Display result: Identified objects

co(True); display(pd.DataFrame(np.sum(post_processing_output)))

#### Function to post records of newly identified objects from frame into a BigQuery database

In [146]:
######################################
# INSERT RECORDS OF NEW OBJECTS INTO BIGQUERY DATABASE

from google.cloud import bigquery
import numpy as np, pandas as pd
from IPython.display import clear_output as co

# set up the BigQuery client using the service account key file
key_path = '../../../../../Apps/APIs/octa-api/credentials/octacity-iduff.json'  # Replace with the path to your service account key file

# set up the dataset and table ids
dataset_id = 'video_analytics'  # Replace with your dataset ID
table_id = 'objetos_identificados'      # Replace with your table ID

# get the BigQuery client and table instances
client = bigquery.Client.from_service_account_json(key_path)
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

def bigquery_post_new_objects(frame, detections, tracking, new_objects, process_start, process_end):
    
    # initialize list for errors
    errors = []
    
    # if there's any nwe object
    if len(new_objects):
        
        # drop unwanted fields
        for obj in new_objects:
            del obj['track_id']
            del obj['bbox']
        
        # insert records of new objects into BigQuery table
        errors = client.insert_rows(table, new_objects)

        # log errors if any
        if errors:
            print('Error inserting record into BigQuery:', errors)

    # return list with errors
    return {'n_new_objects': len(new_objects), 'n_errors': len(errors), 'errors': errors}
    
    
#### Run pipeline · Example usage

post_processing_output = tracking_reid(
    video_path,
    confidence_threshold=0.5,
    max_frames=60,
    post_processing_function=bigquery_post_new_objects,
    proccess_each=3,
)

# Display result: Errors if any

co(True); display(pd.DataFrame(post_processing_output).T)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
n_new_objects,0,0,5,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
n_errors,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
errors,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]


---
### Extra: Google BigQuery table · Set up and usage

In [223]:
# !pip install google-cloud-bigquery

from google.cloud import bigquery

# Set up the BigQuery client using the service account key file
key_path = '../../../../../Apps/APIs/octa-api/credentials/octacity-iduff.json'  # Replace with the path to your service account key file

# Set up the dataset and table information
dataset_id = 'video_analytics'  # Replace with your dataset ID
table_id = 'objetos_identificados'      # Replace with your table ID

# Set BigQuery configuration object
bq_config = {
    'key_path': key_path,
    'dataset_id': dataset_id,
    'table_id': table_id
}

# CREATE TABLE IF IT DOESN'T EXIST

# Define the schema of the table
schema = [
    bigquery.SchemaField('class_label', 'STRING'),
    bigquery.SchemaField('class_name', 'STRING'),
    bigquery.SchemaField('confidence', 'FLOAT'),
    bigquery.SchemaField('timestamp', 'TIMESTAMP'),
    bigquery.SchemaField('url', 'STRING'),
]

# Create the table if it doesn't exist
client = bigquery.Client.from_service_account_json(key_path)
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)

# INSERT ROWS INTO TABLE

# # Insert the rows into the table
# errors = client.insert_rows_json(table, [{}, {}, ...])

# if errors == []:
#     print('Records inserted successfully.')
# else:
#     print(f'Errors occurred during insertion: {errors}')

#### Create `users` database in bigquery

In [2]:
from google.cloud import bigquery

# Set up the BigQuery client using the service account key file
key_path = '../../../../../Apps/APIs/octa-api/credentials/octacity-iduff.json'  # Replace with the path to your service account key file

# Initialize the BigQuery client
client = bigquery.Client.from_service_account_json(key_path)

# Define the schema for the users table
schema = [
    bigquery.SchemaField("email", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("password", "STRING", mode="REQUIRED")
]

# Create the users table in BigQuery
table_id = "octacity.video_analytics.users"  # Replace with your project ID and dataset ID
table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)

print(f"Table {table_id} created successfully.")


Table octacity.video_analytics.users created successfully.


#### Delete all rows from bigquery table

In [ ]:
from google.cloud import bigquery

def delete_all_rows(dataset_id, table_id, key_path):
    # Instantiate the BigQuery client
    client = bigquery.Client.from_service_account_json(key_path)

    # Get the table reference
    table_ref = client.dataset(dataset_id).table(table_id)

    # Get the table object
    table = client.get_table(table_ref)

    # Create a delete query to remove all rows
    delete_query = f"DELETE FROM `{table.project}.{table.dataset_id}.{table.table_id}` WHERE TRUE"
    # Create the job config
    job_config = bigquery.QueryJobConfig()

    # Start the query job
    query_job = client.query(delete_query, job_config=job_config)

    # Wait for the query job to complete
    query_job.result()

    print(f"All rows deleted from {table.project}.{table.dataset_id}.{table.table_id}")

# Example usage
dataset_id = 'video_analytics'  # Your dataset ID
table_id = 'objetos_identificados'      # Your table ID
key_path = '../../../../../Apps/APIs/octa-api/credentials/octacity-iduff.json'  # Replace with the path to your service account key file

delete_all_rows(dataset_id, table_id, key_path)


---
### Extra: Local SQL database file · Set up and usage

In [ ]:
import sqlite3

# Connect to the database or create a new one if it doesn't exist
conn = sqlite3.connect('your_database.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Create a table to store records
cursor.execute('''
    CREATE TABLE IF NOT EXISTS records (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        track_id INTEGER,
        class_label INTEGER,
        class_name TEXT,
        confidence REAL,
        frame_number INTEGER
    )
''')

# Example record data
record_data = {
    'track_id': 1,
    'class_label': 0,
    'class_name': 'Person',
    'confidence': 0.95,
    'frame_number': 10
}

# Insert a record into the table
cursor.execute('''
    INSERT INTO records (track_id, class_label, class_name, confidence, frame_number)
    VALUES (:track_id, :class_label, :class_name, :confidence, :frame_number)
''', record_data)

# Save the changes to the database
conn.commit()

# Query the records from the table
cursor.execute('SELECT * FROM records')
records = cursor.fetchall()

# Print the records
for record in records:
    print(record)

# Close the cursor and the connection
cursor.close()
conn.close()